In [1]:
from tensorflow.python.compiler.tensorrt.trt_convert import TrtPrecisionMode
from tensorflow.experimental.tensorrt import ConversionParams, Converter

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import os
import sys
import shutil
import random
import time
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
sys.path.append(os.path.join(PROJ_DIR, "src"))

import SharedArray as sa
import numpy as np
import time

from pkg.utils.utils import *

args = DummyObject()
args.model_path = "None"
args.rtype="panda"
args.precision= TrtPrecisionMode.FP32

if args.model_path == "None":
    args.model_path = None
PRECISION = args.precision

from pkg.utils.utils_python3 import *
from pkg.controller.robot_config import RobotType
from pkg.planning.filtering.lattice_model.data_utils import *
import numpy as np
int2rtypename = {v.value:v.name for v in RobotType}
DATA_PATH = os.path.join(PROJ_DIR, "data")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

ARM_FOLDER = ARM10_FOLDER
GRASP_SHAPE = (20,20,20)
ARM_SHAPE = (20,20,20)
RH_MASK_SIZE = 512
RH_MASK_STEP = 64

BATCH_SIZE = 1
SERVER_PERIOD = 1e-3

In [3]:
self, prepared_p = DummyObject, sa.create(f"shm://{args.rtype}.prepared", (1,), dtype=np.bool)
self.ROBOT_TYPE_NAME=args.rtype

grasp_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img", (BATCH_SIZE,) + GRASP_SHAPE + (3,))
arm_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.arm_img", (BATCH_SIZE,) + ARM_SHAPE + (1,))
rh_vals_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals", (BATCH_SIZE, 2))
result_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.result", (BATCH_SIZE, 2))
query_in = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_in", (1,), dtype=np.bool)
response_out = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.response_out", (1,), dtype=np.bool)
query_quit = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_quit", (1,), dtype=np.bool)
gtimer = GlobalTimer.instance()
gtimer.reset()
grasp_img_p[:] = 0
arm_img_p[:] = 0
rh_vals_p[:] = 0
result_p[:] = 0
query_in[0] = False
response_out[0] = False
query_quit[0] = False
rh_mask = np.zeros((BATCH_SIZE, 54))

In [4]:
ROBOT_TYPE_NAME=self.ROBOT_TYPE_NAME
ROBOT_MODEL_ROOT = os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
last_save = sorted([item for item in os.listdir(os.path.join(ROBOT_MODEL_ROOT, last_model)) if item.startswith("model")])[-1]
model_path_rel = os.path.join(last_model, last_save)
model_log_dir = os.path.join(ROBOT_MODEL_ROOT, model_path_rel)
model_log_dir_trt = os.path.join(ROBOT_MODEL_ROOT, model_path_rel.replace("model", "trt")+"-"+PRECISION)

### Load saved converted model

In [5]:
# model_k = tf.keras.models.load_model(model_log_dir)
# model_s = tf.saved_model.load(model_log_dir, 
#                               tags=[tf.saved_model.SERVING])
    
# converted_k = tf.keras.models.load_model(model_log_dir_trt)
    
converted_s = tf.saved_model.load(
    model_log_dir_trt, tags=[tf.saved_model.SERVING])


# @tf.function
# def inference_mk(grasp_img_t, arm_img_t, rh_mask_t):
#     # training=False is only needed if there are layers with different
#     # behavior during training versus inference (e.g. Dropout).
#     predictions = model_k([grasp_img_t, arm_img_t, rh_mask_t], training=False)
#     return predictions

# @tf.function
# def inference_ms(grasp_img_t, arm_img_t, rh_mask_t):
#     # training=False is only needed if there are layers with different
#     # behavior during training versus inference (e.g. Dropout).
#     predictions = model_s([grasp_img_t, arm_img_t, rh_mask_t], training=False)
#     return predictions


# @tf.function
# def inference_ck(grasp_img_t, arm_img_t, rh_mask_t):
#     # training=False is only needed if there are layers with different
#     # behavior during training versus inference (e.g. Dropout).
#     predictions = converted_k([grasp_img_t, arm_img_t, rh_mask_t])
#     return predictions

# @tf.function
# def inference_cs(grasp_img_t, arm_img_t, rh_mask_t):
#     # training=False is only needed if there are layers with different
#     # behavior during training versus inference (e.g. Dropout).
#     predictions = converted_s([grasp_img_t, arm_img_t, rh_mask_t])
#     return predictions

## original single run

In [6]:
# gtimer.reset()
# for _ in range(10):
#     with gtimer.block("convert_tf"):
#         query_in[0] = False
#         ## TODO: inference depending on robot type
#         r_mask = div_r_gaussian(rh_vals_p[0][0])
#         h_mask = div_h_gaussian(rh_vals_p[0][1])
#         rh_mask[0] = np.concatenate([r_mask, h_mask])
#         grasp_img_t, arm_img_t, rh_mask_t = (
#             tf.constant(grasp_img_p, dtype=tf.float32), 
#             tf.constant(arm_img_p, dtype=tf.float32), 
#             tf.constant(rh_mask, dtype=tf.float32))
# #     with gtimer.block("model_k"):
# #         result_mk = model_k([grasp_img_t, arm_img_t, rh_mask_t], training=False)
# #     with gtimer.block("inference_mk"):
# #         result_mki = inference_mk(grasp_img_t, arm_img_t, rh_mask_t)
# #     with gtimer.block("model_s"):
# #         result_ms = model_s([grasp_img_t, arm_img_t, rh_mask_t], training=False)
# #     with gtimer.block("inference_ms"):
# #         result_msi = inference_ms(grasp_img_t, arm_img_t, rh_mask_t)
# #     with gtimer.block("converted_k"):
# #         result_ck = converted_k([grasp_img_t, arm_img_t, rh_mask_t])
# #     with gtimer.block("inference_ck"):
# #         result_cki = inference_ck(grasp_img_t, arm_img_t, rh_mask_t)
#     with gtimer.block("converted_s"):
#         result_cs = converted_s([grasp_img_t, arm_img_t, rh_mask_t])
#     with gtimer.block("inference_cs"):
#         result_csi = inference_cs(grasp_img_t, arm_img_t, rh_mask_t)
#     for i_b in range(BATCH_SIZE):
#         result_p[i_b] = result_csi[i_b]
#     response_out[0] = True

# print(gtimer)
# # print("result_mk: \t", result_mk[0].numpy())
# # print("result_mki: \t", result_mki[0].numpy())
# # print("result_ms: \t", result_ms[0].numpy())
# # print("result_msi: \t", result_msi[0].numpy())
# # print("result_ck: \t", result_ck[0].numpy())
# # print("result_cki: \t", result_cki[0].numpy())
# print("result_cs: \t", result_cs[0].numpy())
# print("result_csi: \t", result_csi[0].numpy())

##### Result saved
```
convert_tf: 	24.0 ms/10 = 2.387 ms (0.562/17.323)
model_k: 	453.0 ms/10 = 45.301 ms (42.455/48.52)
inference_mk: 	281.0 ms/10 = 28.089 ms (24.755/32.373)
model_s: 	289.0 ms/10 = 28.858 ms (24.466/34.085)
inference_ms: 	274.0 ms/10 = 27.358 ms (20.217/29.218)
converted_k: 	270.0 ms/10 = 26.963 ms (17.71/30.198)
inference_ck: 	277.0 ms/10 = 27.728 ms (21.448/33.427)
converted_s: 	256.0 ms/10 = 25.561 ms (13.956/31.549)
inference_cs: 	258.0 ms/10 = 25.777 ms (23.357/28.377)

result_mk: 	 [0.9048124 0.8932319]
result_mki: 	 [0.9048124 0.8932319]
result_ms: 	 [0.9048124 0.8932319]
result_msi: 	 [0.9048124 0.8932319]
result_ck: 	 [0.9048124 0.8932319]
result_cki: 	 [0.9048124 0.8932319]
result_cs: 	 [0.9048124 0.8932319]
result_csi: 	 [0.9048124 0.8932319]
```

### convert_variables_to_constants_v2

In [7]:
from tensorflow.python.framework import convert_to_constants

In [8]:
# graph_func_mk = model_k.signatures[
#     tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
# graph_func_ms = model_s.signatures[
#     tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
# graph_func_ck = converted_k.signatures[
#     tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
graph_func_cs = converted_s.signatures[
    tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

In [9]:
# frozen_func_mk = convert_to_constants.convert_variables_to_constants_v2(graph_func_mk)
# frozen_func_ms = convert_to_constants.convert_variables_to_constants_v2(graph_func_ms)
# frozen_func_ck = convert_to_constants.convert_variables_to_constants_v2(graph_func_ck)
frozen_func_cs = convert_to_constants.convert_variables_to_constants_v2(graph_func_cs)

# @tf.function
# def inference_fmk(grasp_img_t, arm_img_t, rh_mask_t):
#     # training=False is only needed if there are layers with different
#     # behavior during training versus inference (e.g. Dropout).
#     predictions = frozen_func_mk(grasp_img_t, arm_img_t, rh_mask_t)
#     return predictions

# @tf.function
# def inference_fms(grasp_img_t, arm_img_t, rh_mask_t):
#     # training=False is only needed if there are layers with different
#     # behavior during training versus inference (e.g. Dropout).
#     predictions = frozen_func_ms(grasp_img_t, arm_img_t, rh_mask_t)
#     return predictions


# @tf.function
# def inference_fck(grasp_img_t, arm_img_t, rh_mask_t):
#     # training=False is only needed if there are layers with different
#     # behavior during training versus inference (e.g. Dropout).
#     predictions = frozen_func_ck(grasp_img_t, arm_img_t, rh_mask_t)
#     return predictions

# @tf.function
# def inference_fcs(grasp_img_t, arm_img_t, rh_mask_t):
#     # training=False is only needed if there are layers with different
#     # behavior during training versus inference (e.g. Dropout).
#     predictions = frozen_func_cs(grasp_img_t, arm_img_t, rh_mask_t)
#     return predictions

## single run

In [11]:
gtimer.reset()
for _ in range(10):
    with gtimer.block("convert_tf"):
        query_in[0] = False
        ## TODO: inference depending on robot type
        r_mask = div_r_gaussian(rh_vals_p[0][0])
        h_mask = div_h_gaussian(rh_vals_p[0][1])
        rh_mask[0] = np.concatenate([r_mask, h_mask])
        grasp_img_t, arm_img_t, rh_mask_t = (
            tf.constant(grasp_img_p, dtype=tf.float32), 
            tf.constant(arm_img_p, dtype=tf.float32), 
            tf.constant(rh_mask, dtype=tf.float32))
#     with gtimer.block("frozen_func_mk"):
#         result_fmk = frozen_func_mk(grasp_img_t, arm_img_t, rh_mask_t)
#     with gtimer.block("inference_fmk"):
#         result_fmki = inference_fmk(grasp_img_t, arm_img_t, rh_mask_t)
#     with gtimer.block("frozen_func_ms"):
#         result_fms = frozen_func_ms(grasp_img_t, arm_img_t, rh_mask_t)
#     with gtimer.block("inference_fms"):
#         result_fmsi = inference_fms(grasp_img_t, arm_img_t, rh_mask_t)
#     with gtimer.block("frozen_func_ck"):
#         result_fck = frozen_func_ck(grasp_img_t, arm_img_t, rh_mask_t)
#     with gtimer.block("inference_fck"):
#         result_fcki = inference_fck(grasp_img_t, arm_img_t, rh_mask_t)
    with gtimer.block("frozen_func_cs"):
        result_fcs = frozen_func_cs(grasp_img_t, arm_img_t, rh_mask_t)
#     with gtimer.block("inference_fcs"):
#         result_fcsi = inference_fcs(grasp_img_t, arm_img_t, rh_mask_t)
    for i_b in range(BATCH_SIZE):
        result_p[i_b] = result_fcs[i_b]
    response_out[0] = True

print(gtimer)
# print("result_fmk: \t", result_fmk[0].numpy())
# print("result_fmki: \t", result_fmki[0].numpy())
# print("result_fms: \t", result_fms[0].numpy())
# print("result_fmsi: \t", result_fmsi[0].numpy())
# print("result_fck: \t", result_fck[0].numpy())
# print("result_fcki: \t", result_fcki[0].numpy())
print("result_fcs: \t", result_fcs[0].numpy())
# print("result_fcsi: \t", result_fcsi[0].numpy())

convert_tf: 	26.0 ms/10 = 2.551 ms (0.715/17.726)
frozen_func_cs: 	236.0 ms/10 = 23.639 ms (20.375/24.467)

result_fcs: 	 [[0.90481395 0.89324564]]


## results log
```
convert_tf: 	8.0 ms/10 = 0.78 ms (0.691/1.137)
frozen_func_mk: 	226.0 ms/10 = 22.627 ms (21.287/25.226)
inference_fmk: 	216.0 ms/10 = 21.615 ms (16.337/24.822)
frozen_func_ms: 	193.0 ms/10 = 19.304 ms (11.226/24.701)
inference_fms: 	220.0 ms/10 = 21.993 ms (19.922/23.384)
frozen_func_ck: 	219.0 ms/10 = 21.94 ms (18.567/24.307)
inference_fck: 	243.0 ms/10 = 24.276 ms (23.317/25.061)
frozen_func_cs: 	215.0 ms/10 = 21.463 ms (18.58/23.41)
inference_fcs: 	229.0 ms/10 = 22.895 ms (21.863/24.178)

result_fmk: 	 [[0.9048124 0.8932319]]
result_fmki: 	 [[0.9048124 0.8932319]]
result_fms: 	 [[0.9048124 0.8932319]]
result_fmsi: 	 [[0.9048124 0.8932319]]
result_fck: 	 [[0.9048144 0.8932461]]
result_fcki: 	 [[0.9048144 0.8932461]]
result_fcs: 	 [[0.90481347 0.89324504]]
result_fcsi: 	 [[0.90481347 0.89324504]]
```

#### low level test

In [15]:
func = graph_func
lower_control_flow=True
aggressive_inlining=False


    converter_data = convert_to_constants._FunctionConverterDataInEager(
          func=func,
          lower_control_flow=lower_control_flow,
          aggressive_inlining=aggressive_inlining)
    output_graph_def, converted_input_indices = convert_to_constants._replace_variables_by_constants(
          converter_data=converter_data)

    fronzen_func = convert_to_constants._construct_concrete_function(func, output_graph_def,
                                                                     converted_input_indices)

## loop test

In [44]:
gtimer=GlobalTimer.instance()
gtimer.reset()
while not query_quit[0]:
    if not query_in[0]:
        time.sleep(SERVER_PERIOD)
        continue
    gtimer.tic("convert_tf")
    query_in[0] = False
    ## TODO: inference depending on robot type
    r_mask = div_r_gaussian(rh_vals_p[0][0])
    h_mask = div_h_gaussian(rh_vals_p[0][1])
    rh_mask[0] = np.concatenate([r_mask, h_mask])
    grasp_img_t, arm_img_t, rh_mask_t = (
        tf.constant(grasp_img_p, dtype=tf.float32), 
        tf.constant(arm_img_p, dtype=tf.float32), 
        tf.constant(rh_mask, dtype=tf.float32))
    etime_ctf = gtimer.toc("convert_tf")
    print("convertin : {} ms".format(round(etime_ctf, 2)))
    gtimer.tic("inference_0")
    result_inf = inference(grasp_img_t, arm_img_t, rh_mask_t)
    etime_inf = gtimer.toc("inference_0")
    print("inference : {} ms".format(round(etime_inf, 2)))
    gtimer.tic("inference_1")
    result_frz = fronzen_func(grasp_img_t, arm_img_t, rh_mask_t)
    etime_inf = gtimer.toc("inference_1")
    print("frozen inf : {} ms".format(round(etime_inf, 2)))
    for i_b in range(BATCH_SIZE):
        result_p[i_b] = result_inf[i_b]
    response_out[0] = True

convertin : 0.22 ms
inference : 45.58 ms
convertin : 0.18 ms
inference : 35.05 ms
convertin : 0.19 ms
inference : 34.7 ms
convertin : 0.17 ms
inference : 19.09 ms
convertin : 0.17 ms
inference : 22.58 ms
convertin : 0.2 ms
inference : 21.32 ms
convertin : 0.17 ms
inference : 34.26 ms
convertin : 0.2 ms
inference : 30.9 ms
convertin : 0.18 ms
inference : 29.55 ms
convertin : 0.18 ms
inference : 36.96 ms
convertin : 0.19 ms
inference : 37.66 ms
convertin : 0.2 ms
inference : 34.23 ms
convertin : 0.18 ms
inference : 39.5 ms
convertin : 0.19 ms
inference : 33.8 ms
convertin : 0.21 ms
inference : 20.37 ms
convertin : 0.19 ms
inference : 38.65 ms
convertin : 0.19 ms
inference : 20.37 ms
convertin : 0.19 ms
inference : 34.66 ms
convertin : 0.18 ms
inference : 37.12 ms
convertin : 0.19 ms
inference : 34.24 ms
convertin : 0.18 ms
inference : 34.06 ms
convertin : 0.18 ms
inference : 33.27 ms
convertin : 0.18 ms
inference : 29.72 ms
convertin : 0.18 ms
inference : 35.87 ms
convertin : 0.1 ms
infe

In [45]:
print(gtimer)

convert_tf: 	18.0 ms/100 = 0.182 ms (0.09/0.222)
inference: 	3184.0 ms/100 = 31.837 ms (18.843/55.542)



In [4]:
self = DummyObject()
self.ROBOT_TYPE_NAME = "panda"
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.arm_img")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.result")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_in")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.response_out")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_quit")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.prepared")